# Predicting Justice Votes

In [40]:
# Standard imports
import numpy as np
import pandas as pd

# Models
from sklearn.naive_bayes import BernoulliNB

# Import data
import pickle

# Scotus class object
from scotus_class import scotus

In [9]:
# Binary
f = open('adj_df.p', 'rb')
df1 = pickle.load(f)
f.close()

# Multiclass
f = open('mul_df.p', 'rb')
df2 = pickle.load(f)
f.close()

## Initial Test of Predicting Justice Opinions
In this initial test, each case a justice has produced an opinion on is a potential target variable.  Justice Rehnquist being the first justice to retire on the list of justices evaluated is the test case.

In [41]:
# Instantiate scotus object for references
sc_obj = scotus(df1)

In [53]:
# Select Rehnquist
sc_obj.justice_courts('Rehnquist')
print('Court Number(s):', sc_obj.j_courts)

Court Number(s): [0]


In [55]:
# Rehnquist court
rehn_court = sc_obj.courts[0]
print(rehn_court)

['Breyer', 'Ginsburg', 'Kennedy', "O'Connor", 'Rehnquist', 'Scalia', 'Souter', 'Stevens', 'Thomas']


In [83]:
# Rehnquist term
print('First Case:', sc_obj.j_cases['Rehnquist'][0])
print('Last Case:', sc_obj.j_cases['Rehnquist'][1])

First Case: 0
Last Case: 497


In [77]:
current_df = df1.loc[rehn_court, :498]
print('Missing justice opinions:', current_df.isna().sum().sum())

Missing justice opinions: 36


In [82]:
# Drop cases with missing opinions
current_df.dropna(axis=1, inplace=True)
print('Cases with no missing opinions:', len(current_df.columns))

Cases with no missing opinions: 465


In [81]:
current_df

,0,1,2,3,4,5,6,7,8,9,...,488,489,490,491,492,493,494,495,496,497
Breyer,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
Ginsburg,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
Kennedy,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
O'Connor,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
Rehnquist,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
Scalia,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0
Souter,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
Stevens,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
Thomas,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
